In [ ]:
import os
from pathlib import Path

cur_path = Path(os.getcwd())
base_dir = cur_path.parent

dataset_path = os.path.join(base_dir, 'dataset')
train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')
save_dir = os.path.join(base_dir, 'saved_models')

clss = ['0_front', '1_back', '1_front', '2_back', '2_front', '5_front', 'ILU']

train_cls_dirs = [os.path.join(train_dir, c) for c in clss]
test_cls_dirs = [os.path.join(test_dir, c) for c in clss]

In [ ]:
import cv2
import numpy as np

def get_X_y_datas(clss_dirs):
    X = []
    y = []
    for i, dir_path in enumerate(clss_dirs):
        files = sorted(os.listdir(clss_dirs[i]))
        files_paths = [os.path.join(dir_path, f) for f in files]
        
        for file_path in files_paths:
            img_bgr = cv2.imread(file_path, cv2.IMREAD_COLOR)
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            input_img = img_rgb.astype(np.float32)/255.
            
            X.append(input_img)
            y.append(int(i))
            
    return np.array(X), np.array(y).astype(np.uint8)

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

num_classes = 7

X, y = get_X_y_datas(train_cls_dirs)
print(X.shape, y.shape)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers

h = 170 # height
w = 130 # width
c = 3  # channels

model = models.Sequential()

model.add(layers.Conv2D(16, 3, input_shape=(h, w, c,)))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.Conv2D(32, 3))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, 3))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.Conv2D(64, 3))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.Conv2D(128, 3))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

# model.add(layers.Flatten())
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, RMSprop

callbacks_list = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
    ),
    ModelCheckpoint(
        filepath=os.path.join(save_dir, 'hand_sign_3.h5'),
        monitor='val_loss',
        save_best_only=True,
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        verbose=1
    )
]

model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(rotation_range=15,
                         width_shift_range=0.3, 
                         height_shift_range=0.15,
                         shear_range=0.2,
                         zoom_range=[1, 1.5],
                         fill_mode='nearest')

batch_size = 32
n_epochs = 100

history = model.fit(aug.flow(X_train, y_train, batch_size=batch_size),
                    validation_data=aug.flow(X_val, y_val, batch_size=batch_size),
                    steps_per_epoch=len(X_train)//batch_size,
                    validation_steps=len(X_val)//batch_size,
                    epochs=n_epochs, callbacks=callbacks_list)                

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation_loss')
plt.title('Loss')
plt.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

y_preds = model.predict(X_val)
y_preds = np.argmax(y_preds, axis=1)
conf_mx = confusion_matrix(np.argmax(y_val, axis=1), y_preds)
# np.fill_diagonal(conf_mx, 0)

In [ ]:
import itertools
def confusion_matrix_plot(cm, classes, 
                          title='Normalized Confusion Matrix', 
                          normalize=True, 
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
    plt.subplots(1, 1, figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
confusion_matrix_plot(conf_mx, classes=clss)